UNIVERSIDAD DE LA HABANA - MATCOM
Ciencia de la Computación
Estadística - Proyecto Final
Curso 2025-2026



ANÁLISIS ESTADÍSTICO DE JUICIOS CIVILES FEDERALES EN ESTADOS UNIDOS (2010-2024)

Autores:
    - Ronald Alfonso Pérez
    - Abraham Rey Sánchez Amador

Institución: Facultad de Matemática y Computación, Universidad de La Habana


## 1. INTRODUCCIÓN



CONTEXTO DEL PROBLEMA

El sistema judicial civil de Estados Unidos procesa anualmente cientos de miles 
de casos que abarcan disputas contractuales, negligencia médica, derechos civiles, 
propiedad intelectual y litigios corporativos. Estos casos no solo determinan 
compensaciones económicas significativas, sino que también establecen precedentes 
legales y reflejan patrones socioeconómicos del país.

El análisis cuantitativo de datos judiciales permite identificar factores que 
influyen en los resultados de los juicios, revelar disparidades sistémicas y 
proporcionar evidencia empírica para reformas procesales. Sin embargo, la 
complejidad de estos datos —con múltiples jurisdicciones, tipos de casos y 
métodos de resolución— requiere técnicas estadísticas avanzadas para extraer 
conclusiones válidas.

RELEVANCIA DEL ESTUDIO

Este proyecto aborda tres dimensiones fundamentales del litigio civil:

1. Factores Económicos: ¿Qué variables predicen mejor las compensaciones otorgadas?
2. Estructura Latente: ¿Existen patrones ocultos que caracterizan perfiles de casos?
3. Disparidades Procesales: ¿El tipo de juicio afecta sistemáticamente los montos?

Comprender estos aspectos tiene implicaciones prácticas para:
- Abogados (estrategias de litigio y estimación de costos)
- Jueces (gestión eficiente de casos)
- Legisladores (diseño de políticas judiciales basadas en evidencia)
- Académicos (teoría empírica del derecho)

OBJETIVOS DEL PROYECTO

1. Realizar un análisis exploratorio exhaustivo de los datos judiciales
2. Aplicar técnicas estadísticas avanzadas (regresión, PCA, pruebas de hipótesis)
3. Identificar patrones significativos en los resultados de litigios civiles
4. Proporcionar conclusiones fundamentadas en evidencia estadística
5. Evaluar críticamente las limitaciones metodológicas del estudio


## 2. PREGUNTAS DE INVESTIGACIÓN



El presente estudio busca responder tres preguntas fundamentales sobre el 
comportamiento de los juicios civiles federales en Estados Unidos durante el 
período 2010-2024:

PREGUNTA 1: Modelado Predictivo de Compensaciones
¿Qué factores (tipo de caso, duración, honorarios legales, jurisdicción) explican 
mejor el monto económico otorgado en juicios civiles federales?

Técnica estadística: Regresión Lineal Múltiple
Justificación: Permite cuantificar la contribución individual de cada variable 
predictora al monto otorgado, controlando por confusores.

PREGUNTA 2: Caracterización de Perfiles de Casos
¿Existen patrones latentes que permitan caracterizar distintos perfiles de 
juicios civiles a partir de variables económicas y temporales, independientemente 
del resultado del juicio?

Técnica estadística: Análisis de Componentes Principales (PCA)
Justificación: Reduce la dimensionalidad del espacio de variables y revela 
estructuras subyacentes no evidentes en el análisis univariado.

PREGUNTA 3: Disparidades por Tipo de Juicio
¿Existen diferencias estadísticamente significativas en los montos otorgados 
según el tipo de juicio (Jury Trial vs Bench Trial) y cómo han variado estas 
diferencias en el período 2014-2024?

Técnica estadística: Prueba de Mann-Whitney U (no paramétrica)
Justificación: Los montos otorgados presentan distribuciones asimétricas con 
valores extremos, violando supuestos de normalidad requeridos por pruebas 
paramétricas.


## 3. IMPORTACIÓN DE LIBRERÍAS



Manipulación y análisis de datos


In [ ]:
import pandas as pd
import numpy as np


Visualización


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


Configuración de visualizaciones


In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline


Configuración de opciones de pandas


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)


Pruebas estadísticas


In [ ]:
from scipy import stats
from scipy.stats import (
    mannwhitneyu, 
    chi2_contingency, 
    normaltest,
    shapiro,
    levene,
    ttest_ind
)


Modelado y machine learning


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, 
    mean_squared_error, 
    mean_absolute_error
)


Análisis estadístico avanzado


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan


Configuración de reproducibilidad


In [ ]:
np.random.seed(42)


Suprimir advertencias innecesarias


In [ ]:
import warnings
warnings.filterwarnings('ignore')

print("✓ Todas las librerías importadas correctamente")
print(f"✓ Versión de pandas: {pd.__version__}")
print(f"✓ Versión de numpy: {np.__version__}")


## 4. DESCRIPCIÓN DEL DATASET



FUENTE DE DATOS

- Dataset: Federal Civil Litigation Cases (2010-2024)
- Fuente: Especificar fuente - ej. Kaggle
- Período temporal: 2010-2024 (14 años)
- Unidad de análisis: Caso civil federal resuelto

DESCRIPCIÓN DE VARIABLES

El dataset contiene información detallada sobre casos civiles federales en EE.UU.,
organizados en las siguientes categorías:

IDENTIFICACIÓN Y CONTEXTO:
- Case_ID: Identificador único del caso
- Filing_Year: Año de presentación del caso (2010-2024)
- Circuit: Circuito judicial federal (First through Eleventh, D.C., Federal)
- District: Distrito judicial específico dentro del circuito

CARACTERÍSTICAS DEL CASO:
- Case_Type: Tipo de litigio civil (Tort, Contract, Employment, IP, etc.)
- Plaintiff_Type: Tipo de demandante (Individual, Corporation, Government, etc.)
- Defendant_Type: Tipo de demandado (Individual, Corporation, Government, etc.)
- Plaintiff_Counsel: Firma legal representante del demandante
- Defendant_Counsel: Firma legal representante del demandado
- Presiding_Judge: Juez que preside el caso

RESOLUCIÓN Y RESULTADOS:
- Outcome: Resultado del caso (Plaintiff Verdict, Defendant Verdict, Settled, Dismissed)
- Resolution_Method: Método de resolución (Jury Trial, Bench Trial, Settlement, etc.)
- Award_Amount: Monto otorgado en dólares (0 si no hay compensación)
- Appeal_Filed: Si se presentó apelación (Yes/No)

VARIABLES ECONÓMICAS Y TEMPORALES:
- Plaintiff_Legal_Fees: Honorarios legales del demandante (USD)
- Defendant_Legal_Fees: Honorarios legales del demandado (USD)
- Days_to_Resolution: Días desde presentación hasta resolución

NOTA IMPORTANTE SOBRE INFLACIÓN:
Los montos monetarios (Award_Amount, Legal_Fees) abarcan 14 años (2010-2024).
Para hacer comparaciones válidas entre años, todos los valores monetarios serán
ajustados a dólares constantes de 2024 utilizando el Índice de Precios al
Consumidor (CPI) de la Oficina de Estadísticas Laborales de EE.UU.


## 5. CARGA Y EXPLORACIÓN INICIAL DE DATOS



Cargar el dataset


In [ ]:
print("="*80)
print("CARGANDO DATASET...")
print("="*80)

df = pd.read_csv('federal_civil_litigations.csv')

print(f"\n✓ Dataset cargado exitosamente")
print(f"✓ Dimensiones: {df.shape[0]:,} casos × {df.shape[1]} variables")
print(f"✓ Período: {df['Filing_Year'].min()} - {df['Filing_Year'].max()}")


Primeras observaciones


In [ ]:
print("\n" + "="*80)
print("PRIMERAS 5 OBSERVACIONES")
print("="*80)
print(df.head())


Información general del dataset


In [ ]:
print("\n" + "="*80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*80)
print(df.info())


Estadísticos descriptivos de variables numéricas


In [ ]:
print("\n" + "="*80)
print("ESTADÍSTICOS DESCRIPTIVOS - VARIABLES NUMÉRICAS")
print("="*80)
print(df.describe())


Verificar valores duplicados


In [ ]:
duplicados = df.duplicated().sum()
print(f"\n✓ Casos duplicados encontrados: {duplicados}")

if duplicados > 0:
    print(f"  → Eliminando {duplicados} casos duplicados...")
    df = df.drop_duplicates()
    print(f"  ✓ Dataset limpio: {df.shape[0]:,} casos únicos")


Verificar valores faltantes


In [ ]:
print("\n" + "="*80)
print("ANÁLISIS DE VALORES FALTANTES")
print("="*80)

valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (valores_faltantes / len(df)) * 100

missing_df = pd.DataFrame({
    'Variable': valores_faltantes.index,
    'Valores Faltantes': valores_faltantes.values,
    'Porcentaje (%)': porcentaje_faltantes.values
})

missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values(
    'Valores Faltantes', 
    ascending=False
)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("✓ No se encontraron valores faltantes en el dataset")


Distribución de variables categóricas clave


In [ ]:
print("\n" + "="*80)
print("DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS CLAVE")
print("="*80)

print("\n1. DISTRIBUCIÓN POR AÑO:")
print(df['Filing_Year'].value_counts().sort_index())

print("\n2. DISTRIBUCIÓN POR TIPO DE CASO (Top 10):")
print(df['Case_Type'].value_counts().head(10))

print("\n3. DISTRIBUCIÓN POR RESULTADO DEL CASO:")
print(df['Outcome'].value_counts())

print("\n4. DISTRIBUCIÓN POR MÉTODO DE RESOLUCIÓN:")
print(df['Resolution_Method'].value_counts())

print("\n5. DISTRIBUCIÓN POR CIRCUITO (Top 5):")
print(df['Circuit'].value_counts().head(5))


Resumen de variables monetarias (antes del ajuste por inflación)


In [ ]:
print("\n" + "="*80)
print("RESUMEN DE VARIABLES MONETARIAS (Dólares nominales)")
print("="*80)

monetary_vars = ['Award_Amount', 'Plaintiff_Legal_Fees', 'Defendant_Legal_Fees']

for var in monetary_vars:
    print(f"\n{var}:")
    print(f"  - Media: ${df[var].mean():,.2f}")
    print(f"  - Mediana: ${df[var].median():,.2f}")
    print(f"  - Mínimo: ${df[var].min():,.2f}")
    print(f"  - Máximo: ${df[var].max():,.2f}")
    print(f"  - Casos con valor > 0: {(df[var] > 0).sum():,} ({(df[var] > 0).mean()*100:.1f}%)")

print("\n" + "="*80)
print("✓ CARGA Y EXPLORACIÓN INICIAL COMPLETADA")
print("="*80)
